# Demo of Helper Procedure pr\_setDefaultExtendedProperties

This procedure sets default extended properties on undocumented columns of a table or the table itself. It is written so you can run it multiple times....if the property is already present it leaves it alone if not...it is created. 

My default properties are:

- Author (supplied via parameter @paramExtendedPropertyAuthor)
- ChangeDate
- ChangeHistory
- CreationDate (defaults to GETDATE in German Format dd.MM.yyyy)
- MS\_Description

Grab the source code at .The procedure is supposed to be created in the same database where the objects live you would like to document. If you have 10 databases you would need 1 procedure per each database.

Here are a few procedure calls demonstrating the behaviour.

  

First we create a test table in the database of your choice.

In [4]:
DROP TABLE IF EXISTS dbo.test;
GO
CREATE TABLE dbo.test 
(
	a INT NOT NULL IDENTITY(1,1),
	b VARCHAR(20) NULL,
	CONSTRAINT test_pk PRIMARY KEY CLUSTERED (a)
);

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.057

Let's double check that we don't have any extended properties present yet. The following SQL should return 0 rows.

In [5]:
SELECT 
        ob.name AS objectName,
        '0Table' AS columnName,
        ep.name AS propertyName,
        ep.value AS propertyValue
FROM   
           sys.objects ob
INNER JOIN sys.extended_properties ep ON ep.major_id = ob.object_id AND  ep.minor_id = 0
WHERE
      ob.name  = 'test'
AND   ob.type_desc IN ('VIEW', 'USER_TABLE')
AND   ep.class_desc = 'OBJECT_OR_COLUMN'
UNION ALL
SELECT 
        ob.name AS objectName,
        c.name AS columnName,
        ep.name AS propertyName,
        ep.value AS propertyValue
FROM   
           sys.objects ob
INNER JOIN sys.columns c ON c.object_id = ob.object_id               
INNER JOIN sys.extended_properties ep ON ep.major_id = ob.object_id AND  ep.minor_id = c.column_id
WHERE
      ob.name  = 'test'
AND   ob.type_desc IN ('VIEW', 'USER_TABLE')
AND   ep.class_desc = 'OBJECT_OR_COLUMN'
ORDER BY
       objectName,
       columnName
;


(0 rows affected)

Total execution time: 00:00:00.070

objectName,columnName,propertyName,propertyValue


Now execute the procedure.  
Observe the output...the object as well as the column names are listed.

In [6]:
EXECUTE dbo.pr_setDefaultExtendedProperty 
	@paramSchemaNameDestination = 'dbo',
	@paramObjectNameDestination = 'test',
	@paramObjectTypeDestination = 'U', /* user table...if you omit this parameter you get a list of allowed values returned */
	@paramExtendedPropertyAuthor = 'Mickey Mouse'
;

(5 rows affected)

(2 rows affected)

Schema: dbo

Objekt: test

Typ: TABLE

...

Extended Property 'ChangeHistory' Neuer Wert ''

Extended Property 'CreationDate' Neuer Wert '27.09.2023'

Extended Property 'MS_Description' Neuer Wert ''

========================================================================================================================

Total execution time: 00:00:00.237

Check Extended Properties again....now with some PIVOTing magic --\> Extended Properties are present

In [7]:
SELECT 
       objectName,
       columnName,
       [MS_Description],
       [Author],
       [CreationDate],
       [ChangeDate],
       [ChangeHistory]
FROM   
(
  SELECT 
         ob.name AS objectName,
         '0Table' AS columnName,
         ep.name AS propertyName,
         ep.value AS propertyValue
  FROM   
                  sys.objects ob
  LEFT OUTER JOIN sys.extended_properties ep ON ep.major_id = ob.object_id AND  ep.minor_id = 0
  WHERE
         ob.name = 'test'
  AND ob.type_desc IN ('VIEW', 'USER_TABLE')
  AND ep.class_desc = 'OBJECT_OR_COLUMN'
  UNION ALL
  SELECT 
         ob.name AS objectName,
         c.name AS columnName,
         ep.name AS propertyName,
         ep.value AS propertyValue
  FROM   
                  sys.objects ob
  LEFT OUTER JOIN sys.columns c ON c.object_id = ob.object_id               
  LEFT OUTER JOIN sys.extended_properties ep ON ep.major_id = ob.object_id AND  ep.minor_id = c.column_id
  WHERE
         ob.name  = 'test'
         
  AND ob.type_desc IN ('VIEW', 'USER_TABLE')
  AND ep.class_desc = 'OBJECT_OR_COLUMN'
) src
PIVOT (MAX(propertyValue) FOR propertyName IN ([MS_Description], [Author], [CreationDate], [ChangeDate], [ChangeHistory])
) piv

(3 rows affected)

Total execution time: 00:00:00.010

objectName,columnName,MS_Description,Author,CreationDate,ChangeDate,ChangeHistory
test,0Table,,Mickey Mouse,27.09.2023,,
test,a,,Mickey Mouse,27.09.2023,,
test,b,,Mickey Mouse,27.09.2023,,


Now let's add a new column...execute the procedure again and see what happens

In [8]:
ALTER TABLE dbo.test ADD c NUMERIC(10,2);

EXECUTE dbo.pr_setDefaultExtendedProperty 
	@paramSchemaNameDestination = 'dbo',
	@paramObjectNameDestination = 'test',
	@paramObjectTypeDestination = 'U', /* user table...if you omit this parameter you get a list of allowed values returned */
	@paramExtendedPropertyAuthor = 'Donald Duck'
;

(5 rows affected)

(3 rows affected)

Schema: dbo

Objekt: test

Typ: TABLE

...

Extended Property 'ChangeHistory' Neuer Wert ''

Extended Property 'CreationDate' Neuer Wert '27.09.2023'

Extended Property 'MS_Description' Neuer Wert ''

========================================================================================================================

Total execution time: 00:00:00.092

Observe the output: Only for the new column c the values are printed out. The output claims that no action was done for the other columns and the table level.

Let's verify this by checking the extended properties again.

In [9]:
SELECT 
       objectName,
       columnName,
       [MS_Description],
       [Author],
       [CreationDate],
       [ChangeDate],
       [ChangeHistory]
FROM   
(
  SELECT 
         ob.name AS objectName,
         '0Table' AS columnName,
         ep.name AS propertyName,
         ep.value AS propertyValue
  FROM   
                  sys.objects ob
  LEFT OUTER JOIN sys.extended_properties ep ON ep.major_id = ob.object_id AND  ep.minor_id = 0
  WHERE
         ob.name = 'test'
  AND ob.type_desc IN ('VIEW', 'USER_TABLE')
  AND ep.class_desc = 'OBJECT_OR_COLUMN'
  UNION ALL
  SELECT 
         ob.name AS objectName,
         c.name AS columnName,
         ep.name AS propertyName,
         ep.value AS propertyValue
  FROM   
                  sys.objects ob
  LEFT OUTER JOIN sys.columns c ON c.object_id = ob.object_id               
  LEFT OUTER JOIN sys.extended_properties ep ON ep.major_id = ob.object_id AND  ep.minor_id = c.column_id
  WHERE
         ob.name  = 'test'
         
  AND ob.type_desc IN ('VIEW', 'USER_TABLE')
  AND ep.class_desc = 'OBJECT_OR_COLUMN'
) src
PIVOT (MAX(propertyValue) FOR propertyName IN ([MS_Description], [Author], [CreationDate], [ChangeDate], [ChangeHistory])
) piv

(4 rows affected)

Total execution time: 00:00:00.009

objectName,columnName,MS_Description,Author,CreationDate,ChangeDate,ChangeHistory
test,0Table,,Mickey Mouse,27.09.2023,,
test,a,,Mickey Mouse,27.09.2023,,
test,b,,Mickey Mouse,27.09.2023,,
test,c,,Donald Duck,27.09.2023,,


Author Donald Duck has only been set for the new column c as expected.

You have trouble putting the right object type?..we got you covered...if you leave it out you get an error message listing the valid values:

In [12]:

EXECUTE dbo.pr_setDefaultExtendedProperty 
	@paramSchemaNameDestination = 'dbo',
	@paramObjectNameDestination = 'test',
	@paramObjectTypeDestination = NULL,
	@paramExtendedPropertyAuthor = 'Donald Duck'
;

Parameter @paramObjectTypeDestination hat eine falsche Eingabe.

Unterstützte Eingaben sind:

U für User Table,

V für View,

P für SQL_STORED_PROCEDURE,

...

TF für SQL_TABLE_VALUED_FUNCTION,

IN für SQL_INLINE_TABLE_VALUED_FUNCTION,

TR für TRIGGER angeben.

TT für Tabellentyp angeben.

Total execution time: 00:00:00.036

In [14]:
/* finally let's cleanup the demo table */
IF OBJECT_ID('dbo.test') IS NOT NULL
    DROP TABLE dbo.test;

Commands completed successfully.

Total execution time: 00:00:00.003